In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import time

# Add Matplotlib inline magic command
%matplotlib inline

In [2]:
# Reading in all CSVs as DataFrames
fire_df = pd.read_csv("fire_copy.csv")
housing_df = pd.read_csv("housing.csv")
electricity_df = pd.read_csv("electricity.csv")
environment_df = pd.read_csv("environment_data_june24.csv")

# Cleaning up Fire Data Set

In [3]:
# Taking a look at fire_df
fire_df.sample(n=5)

,Unnamed: 0,AcresBurned,ArchiveYear,CalFireIncident,Counties,CountyIds,Latitude,Longitude,MajorIncident,Name,PersonnelInvolved,Started
244,244,75.0,2014,True,Shasta,45,40.571900,-121.341500,True,Cassel Fire,108.0,2014-07-05T18:44:00Z
709,709,690.0,2017,True,Tuolumne,55,37.905545,-120.408135,True,Jacksonville Fire,NaN,2017-07-29T13:50:00Z
125,125,44.0,2013,False,Ventura,56,0.000000,0.000000,False,Happy Camp Fire,NaN,2013-12-10T02:26:00Z
245,245,75.0,2014,True,Monterey,27,36.103000,-121.117000,False,San Lucas Fire,NaN,2014-04-30T14:25:00Z
1240,1240,60.0,2018,True,Madera,20,37.146630,-119.912230,False,Raymond Fire,NaN,2018-06-15T11:46:00Z


In [4]:
# Modify column names in fire_df
fire_df_new_columns = ["Unnamed: 0", "Acres Burned", "Year", "Cal Fire Incident", "County", "County IDs", "Latitude", "Longitude", 
                       "Major Incident", "Name", "Personnel Involved", "Started"]

fire_df.columns = fire_df_new_columns
fire_df.head()

,Unnamed: 0,Acres Burned,Year,Cal Fire Incident,County,County IDs,Latitude,Longitude,Major Incident,Name,Personnel Involved,Started
0,0,257314.0,2013,True,Tuolumne,55,37.857000,-120.086000,False,Rim Fire,NaN,2013-08-17T15:25:00Z
1,1,30274.0,2013,True,Los Angeles,19,34.585595,-118.423176,False,Powerhouse Fire,NaN,2013-05-30T15:28:00Z
2,2,27531.0,2013,True,Riverside,33,33.709500,-116.728850,False,Mountain Fire,NaN,2013-07-15T13:43:00Z
3,3,27440.0,2013,False,Placer,31,39.120000,-120.650000,False,American Fire,NaN,2013-08-10T16:30:00Z
4,4,24251.0,2013,True,Ventura,56,0.000000,0.000000,True,Springs Fire,2167.0,2013-05-02T07:01:00Z


In [5]:
# Fill 0's instead of NaN in the rows of Acres Burned column
fire_df["Acres Burned"] = fire_df["Acres Burned"].fillna(0)

In [6]:
# Format Latitude and Longitude columns in fire_df
fire_df["Latitude"] = fire_df["Latitude"].map("{:.2f}".format)
fire_df["Longitude"] = fire_df["Longitude"].map("{:.2f}".format)
fire_df.head()

,Unnamed: 0,Acres Burned,Year,Cal Fire Incident,County,County IDs,Latitude,Longitude,Major Incident,Name,Personnel Involved,Started
0,0,257314.0,2013,True,Tuolumne,55,37.86,-120.09,False,Rim Fire,NaN,2013-08-17T15:25:00Z
1,1,30274.0,2013,True,Los Angeles,19,34.59,-118.42,False,Powerhouse Fire,NaN,2013-05-30T15:28:00Z
2,2,27531.0,2013,True,Riverside,33,33.71,-116.73,False,Mountain Fire,NaN,2013-07-15T13:43:00Z
3,3,27440.0,2013,False,Placer,31,39.12,-120.65,False,American Fire,NaN,2013-08-10T16:30:00Z
4,4,24251.0,2013,True,Ventura,56,0.00,0.00,True,Springs Fire,2167.0,2013-05-02T07:01:00Z


In [7]:
# Convert "Latitude" and "Longitude" into floats, "County IDs" and "Acres Burned" into int
fire_df["Latitude"] = fire_df["Latitude"].astype(float)
fire_df["Longitude"] = fire_df["Longitude"].astype(float)
fire_df["County IDs"] = fire_df["County IDs"].astype(int)
fire_df["Acres Burned"] = fire_df["Acres Burned"].astype(int)
fire_df["Cal Fire Incident"] = fire_df["Cal Fire Incident"].astype(str)
fire_df["Major Incident"] = fire_df["Major Incident"].astype(str)

In [8]:
# Modify "Started" column in fire_df from object to datetime
fire_df["Started"] = pd.to_datetime(fire_df["Started"], infer_datetime_format=True)
fire_df.head()

,Unnamed: 0,Acres Burned,Year,Cal Fire Incident,County,County IDs,Latitude,Longitude,Major Incident,Name,Personnel Involved,Started
0,0,257314,2013,True,Tuolumne,55,37.86,-120.09,False,Rim Fire,NaN,2013-08-17 15:25:00+00:00
1,1,30274,2013,True,Los Angeles,19,34.59,-118.42,False,Powerhouse Fire,NaN,2013-05-30 15:28:00+00:00
2,2,27531,2013,True,Riverside,33,33.71,-116.73,False,Mountain Fire,NaN,2013-07-15 13:43:00+00:00
3,3,27440,2013,False,Placer,31,39.12,-120.65,False,American Fire,NaN,2013-08-10 16:30:00+00:00
4,4,24251,2013,True,Ventura,56,0.00,0.00,True,Springs Fire,2167.0,2013-05-02 07:01:00+00:00


### Since "Started" and "Archive Year" have the same years, should we get rid of 'Archive Year'
### since it's redundant and may overfit the model? Plus it can't be converted into datetime? --> Actually keep Archive Year so it's easier to group with other datasets that just have the year

In [9]:
# Drop "Unnamed: 0" and "Personnel Involved" columns from fire_df
fire_df.drop(["Unnamed: 0", "Personnel Involved"], axis=1, inplace=True)

In [10]:
# Check data types to see if they have been properly updated
fire_df.dtypes

Acres Burned                       int64
Year                               int64
Cal Fire Incident                 object
County                            object
County IDs                         int64
Latitude                         float64
Longitude                        float64
Major Incident                    object
Name                              object
Started              datetime64[ns, UTC]
dtype: object

In [11]:
# Reorder fire_df columns
fire_df = fire_df[["Name", "County", "County IDs", "Latitude", "Longitude", "Started",
                  "Year", "Acres Burned", "Cal Fire Incident", "Major Incident"]]
fire_df.head()

,Name,County,County IDs,Latitude,Longitude,Started,Year,Acres Burned,Cal Fire Incident,Major Incident
0,Rim Fire,Tuolumne,55,37.86,-120.09,2013-08-17 15:25:00+00:00,2013,257314,True,False
1,Powerhouse Fire,Los Angeles,19,34.59,-118.42,2013-05-30 15:28:00+00:00,2013,30274,True,False
2,Mountain Fire,Riverside,33,33.71,-116.73,2013-07-15 13:43:00+00:00,2013,27531,True,False
3,American Fire,Placer,31,39.12,-120.65,2013-08-10 16:30:00+00:00,2013,27440,False,False
4,Springs Fire,Ventura,56,0.00,0.00,2013-05-02 07:01:00+00:00,2013,24251,True,True


# ---------------------------------------------------------------------

# Cleaning up Housing Data Set

In [12]:
# Taking a look at housing_df
housing_df.sample(n=5)

,Unnamed: 0,longitude,latitude,population,households,median_income,median_house_value,ocean_proximity,County
9819,9819,-117.86,33.62,2339.0,743.0,9.06780,500001.0,<1H OCEAN,Orange County
5142,5142,-120.49,37.26,1283.0,378.0,1.89390,83000.0,INLAND,Merced County
6571,6571,-119.04,35.36,1822.0,717.0,2.43985,68200.0,INLAND,Kern County
6234,6234,-119.31,36.34,3994.0,1032.0,1.81655,59650.0,INLAND,Tulare County
178,178,-123.23,39.13,529.0,217.0,3.89580,144000.0,<1H OCEAN,Mendocino County


In [13]:
# Modify column names in housing_df
housing_df_new_columns = ["Unnamed: 0", "Longitude", "Latitude", "Population", "Households", 
                          "Average Income", "Average House Value", "Ocean Proximity", "County"]

housing_df.columns = housing_df_new_columns
housing_df.head()

,Unnamed: 0,Longitude,Latitude,Population,Households,Average Income,Average House Value,Ocean Proximity,County
0,0,-124.35,40.54,806.0,270.0,3.0147,94600.0,NEAR OCEAN,Humboldt County
1,1,-124.30,41.80,1298.0,478.0,1.9797,85800.0,NEAR OCEAN,Del Norte County
2,2,-124.30,41.84,1244.0,456.0,3.0313,103600.0,NEAR OCEAN,Del Norte County
3,3,-124.27,40.69,1194.0,465.0,2.5179,79000.0,NEAR OCEAN,Humboldt County
4,4,-124.26,40.58,907.0,369.0,2.3571,111400.0,NEAR OCEAN,Humboldt County


In [14]:
# Split the word "County" from every row in "County" column
housing_df["County"] = housing_df["County"].str.split("County", n=1, expand=True)

In [15]:
# Check to see if it worked
housing_df["County"]

0              Humboldt 
1             Del Norte 
2             Del Norte 
3              Humboldt 
4              Humboldt 
              ...       
12585         Riverside 
12586          Imperial 
12587            La Paz 
12588    San Bernardino 
12589    San Bernardino 
Name: County, Length: 12590, dtype: object

In [16]:
# Check to see if housing_df has newly updated "County" column
housing_df.head()

,Unnamed: 0,Longitude,Latitude,Population,Households,Average Income,Average House Value,Ocean Proximity,County
0,0,-124.35,40.54,806.0,270.0,3.0147,94600.0,NEAR OCEAN,Humboldt
1,1,-124.30,41.80,1298.0,478.0,1.9797,85800.0,NEAR OCEAN,Del Norte
2,2,-124.30,41.84,1244.0,456.0,3.0313,103600.0,NEAR OCEAN,Del Norte
3,3,-124.27,40.69,1194.0,465.0,2.5179,79000.0,NEAR OCEAN,Humboldt
4,4,-124.26,40.58,907.0,369.0,2.3571,111400.0,NEAR OCEAN,Humboldt


In [17]:
# Adjusting "Average Income" column so that each value is multipled by 10,000
housing_df["Average Income"] = housing_df["Average Income"].multiply(other=10**4)
housing_df.head()

,Unnamed: 0,Longitude,Latitude,Population,Households,Average Income,Average House Value,Ocean Proximity,County
0,0,-124.35,40.54,806.0,270.0,30147.0,94600.0,NEAR OCEAN,Humboldt
1,1,-124.30,41.80,1298.0,478.0,19797.0,85800.0,NEAR OCEAN,Del Norte
2,2,-124.30,41.84,1244.0,456.0,30313.0,103600.0,NEAR OCEAN,Del Norte
3,3,-124.27,40.69,1194.0,465.0,25179.0,79000.0,NEAR OCEAN,Humboldt
4,4,-124.26,40.58,907.0,369.0,23571.0,111400.0,NEAR OCEAN,Humboldt


In [18]:
# Format Population, Households, Average Income, and Average House Value columns in housing_df
housing_df["Population"] = housing_df["Population"].map("{:.0f}".format)
housing_df["Households"] = housing_df["Households"].map("{:.0f}".format)
housing_df["Average Income"] = housing_df["Average Income"].map("{:.0f}".format)
housing_df["Average House Value"] = housing_df["Average House Value"].map("{:.0f}".format)
housing_df.head()

,Unnamed: 0,Longitude,Latitude,Population,Households,Average Income,Average House Value,Ocean Proximity,County
0,0,-124.35,40.54,806,270,30147,94600,NEAR OCEAN,Humboldt
1,1,-124.30,41.80,1298,478,19797,85800,NEAR OCEAN,Del Norte
2,2,-124.30,41.84,1244,456,30313,103600,NEAR OCEAN,Del Norte
3,3,-124.27,40.69,1194,465,25179,79000,NEAR OCEAN,Humboldt
4,4,-124.26,40.58,907,369,23571,111400,NEAR OCEAN,Humboldt


In [19]:
# Change Population, Households, Average Income, and Average House Value to integers
housing_df["Population"] = housing_df["Population"].astype(int)
housing_df["Households"] = housing_df["Households"].astype(int)
housing_df["Average Income"] = housing_df["Average Income"].astype(int)
housing_df["Average House Value"] = housing_df["Average House Value"].astype(int)

In [20]:
# Drop "Unnamed: 0" column from housing_df
housing_df.drop("Unnamed: 0", axis=1, inplace=True)

In [21]:
# Check housing_df data types to see if they were successfully converted
housing_df.dtypes

Longitude              float64
Latitude               float64
Population               int64
Households               int64
Average Income           int64
Average House Value      int64
Ocean Proximity         object
County                  object
dtype: object

In [22]:
# Reorder housing_df columns
housing_df = housing_df[["County", "Latitude", "Longitude", "Ocean Proximity", "Population", 
               "Households", "Average Income", "Average House Value"]]

housing_df.head()

,County,Latitude,Longitude,Ocean Proximity,Population,Households,Average Income,Average House Value
0,Humboldt,40.54,-124.35,NEAR OCEAN,806,270,30147,94600
1,Del Norte,41.80,-124.30,NEAR OCEAN,1298,478,19797,85800
2,Del Norte,41.84,-124.30,NEAR OCEAN,1244,456,30313,103600
3,Humboldt,40.69,-124.27,NEAR OCEAN,1194,465,25179,79000
4,Humboldt,40.58,-124.26,NEAR OCEAN,907,369,23571,111400


In [23]:
# Take away white space from right side of County names
housing_df["County"] = housing_df["County"].str.rstrip()

housing_df["County"].unique().tolist()

['Humboldt',
 'Del Norte',
 nan,
 'Mendocino',
 'Trinity',
 'Sonoma',
 'Siskiyou',
 'Lake',
 'Marin',
 'Shasta',
 'Glenn',
 'Tehama',
 'Napa',
 'San Mateo',
 'San Francisco',
 'Colusa',
 'Contra Costa',
 'Alameda',
 'Solano',
 'Santa Cruz',
 'Yolo',
 'Santa Clara',
 'Butte',
 'Monterey',
 'Sutter',
 'Sacramento',
 'San Benito',
 'Yuba',
 'San Joaquin',
 'Placer',
 'Stanislaus',
 'Nevada',
 'Plumas',
 'Modoc',
 'San Luis Obispo',
 'Lassen',
 'El Dorado',
 'Merced',
 'Amador',
 'Sierra',
 'Calaveras',
 'Fresno',
 'Santa Barbara',
 'Tuolumne',
 'Madera',
 'Mariposa',
 'Kings',
 'Alpine',
 'Douglas',
 'Kern',
 'Tulare',
 'Mono',
 'Ventura',
 'Los Angeles',
 'Inyo',
 'Orange',
 'San Bernardino',
 'Riverside',
 'San Diego',
 'Imperial',
 'La Paz']

In [24]:
# Check housing_df
housing_df.head()

,County,Latitude,Longitude,Ocean Proximity,Population,Households,Average Income,Average House Value
0,Humboldt,40.54,-124.35,NEAR OCEAN,806,270,30147,94600
1,Del Norte,41.80,-124.30,NEAR OCEAN,1298,478,19797,85800
2,Del Norte,41.84,-124.30,NEAR OCEAN,1244,456,30313,103600
3,Humboldt,40.69,-124.27,NEAR OCEAN,1194,465,25179,79000
4,Humboldt,40.58,-124.26,NEAR OCEAN,907,369,23571,111400


# ---------------------------------------------------------------------

# Cleaning up Electricity Data Set

In [25]:
# Taking a look at electricity_df
electricity_df.sample(n=5)

,Unnamed: 0,County,2019,2018,2017,2016,2015,2014,2013
55,167,VENTURA,5344.035137,5446.199174,5508.925934,5455.495304,5591.098091,5486.039211,5397.690876
36,110,SAN DIEGO,19047.674168,19732.624780,19667.445537,19665.677640,19894.493440,19998.953248,19687.538189
18,56,LOS ANGELES,66118.673005,67907.482625,68656.720182,69414.044656,69532.293389,69953.419146,68373.298413
38,116,SAN JOAQUIN,5583.337799,5658.477632,5626.568060,5430.762061,5141.428870,5211.026533,5524.571740
35,107,SAN BERNARDINO,14987.210320,15371.557356,15283.329154,14946.910952,14732.329242,14732.812970,14375.326790


In [26]:
# Check electricity_df data types
electricity_df.dtypes

Unnamed: 0      int64
County         object
2019          float64
2018          float64
2017          float64
2016          float64
2015          float64
2014          float64
2013          float64
dtype: object

In [27]:
# Convert all rows in County Column from uppercase to normal writing
electricity_df["County"] = electricity_df["County"].str.title()
electricity_df["County"]

0             Alameda
1              Alpine
2              Amador
3               Butte
4           Calaveras
5              Colusa
6        Contra Costa
7           Del Norte
8           El Dorado
9              Fresno
10              Glenn
11           Humboldt
12           Imperial
13               Inyo
14               Kern
15              Kings
16               Lake
17             Lassen
18        Los Angeles
19             Madera
20              Marin
21           Mariposa
22          Mendocino
23             Merced
24              Modoc
25               Mono
26           Monterey
27               Napa
28             Nevada
29             Orange
30             Placer
31             Plumas
32          Riverside
33         Sacramento
34         San Benito
35     San Bernardino
36          San Diego
37      San Francisco
38        San Joaquin
39    San Luis Obispo
40          San Mateo
41      Santa Barbara
42        Santa Clara
43         Santa Cruz
44             Shasta
45        

In [28]:
# Check to see if all the counties are now in normal writing in County column
electricity_df.head()

,Unnamed: 0,County,2019,2018,2017,2016,2015,2014,2013
0,2,Alameda,10684.085867,10391.361826,11079.450563,10791.224841,10235.384987,10299.877787,10618.661255
1,5,Alpine,18.906214,18.704208,18.976912,17.415654,16.209900,15.983364,18.176468
2,8,Amador,317.885054,304.092677,313.312866,309.094100,285.308438,289.501492,310.344276
3,11,Butte,1396.246344,1475.788821,1529.818607,1482.073760,1492.098630,1489.484147,1502.980505
4,14,Calaveras,330.559070,332.353511,347.931437,316.143601,311.620104,319.671411,326.656878


In [29]:
# Format the numbers in all the year columns in electricity_df
electricity_df["2019"] = electricity_df["2019"].map("{:.2f}".format)
electricity_df["2018"] = electricity_df["2018"].map("{:.2f}".format)
electricity_df["2017"] = electricity_df["2017"].map("{:.2f}".format)
electricity_df["2016"] = electricity_df["2016"].map("{:.2f}".format)
electricity_df["2015"] = electricity_df["2015"].map("{:.2f}".format)
electricity_df["2014"] = electricity_df["2014"].map("{:.2f}".format)
electricity_df["2013"] = electricity_df["2013"].map("{:.2f}".format)

In [30]:
# Convert all the years back into integers in electricity_df
electricity_df["2019"] = electricity_df["2019"].astype(float)
electricity_df["2018"] = electricity_df["2018"].astype(float)
electricity_df["2017"] = electricity_df["2017"].astype(float)
electricity_df["2016"] = electricity_df["2016"].astype(float)
electricity_df["2015"] = electricity_df["2015"].astype(float)
electricity_df["2014"] = electricity_df["2014"].astype(float)
electricity_df["2013"] = electricity_df["2013"].astype(float)

In [31]:
# Check to see if columns were successfully converted to floats in electricity_df
electricity_df.dtypes

Unnamed: 0      int64
County         object
2019          float64
2018          float64
2017          float64
2016          float64
2015          float64
2014          float64
2013          float64
dtype: object

In [32]:
# Drop "Unnamed: 0" column from electricity_df
electricity_df.drop("Unnamed: 0", axis=1, inplace=True)

In [33]:
# Reorder columns in electricity_df 
electricity_df = electricity_df[["County", "2013", "2014", "2015", "2016", "2017", "2018", "2019"]]
electricity_df.head()

,County,2013,2014,2015,2016,2017,2018,2019
0,Alameda,10618.66,10299.88,10235.38,10791.22,11079.45,10391.36,10684.09
1,Alpine,18.18,15.98,16.21,17.42,18.98,18.70,18.91
2,Amador,310.34,289.50,285.31,309.09,313.31,304.09,317.89
3,Butte,1502.98,1489.48,1492.10,1482.07,1529.82,1475.79,1396.25
4,Calaveras,326.66,319.67,311.62,316.14,347.93,332.35,330.56


In [34]:
# Make Years into a column
electricity_df = electricity_df.melt(id_vars="County", var_name="Year", value_name="MWh")
electricity_df

,County,Year,MWh
0,Alameda,2013,10618.66
1,Alpine,2013,18.18
2,Amador,2013,310.34
3,Butte,2013,1502.98
4,Calaveras,2013,326.66
...,...,...,...
401,Tulare,2019,4162.20
402,Tuolumne,2019,452.64
403,Ventura,2019,5344.04
404,Yolo,2019,1720.75


In [35]:
# # Add MWh to each of the column names since values are in millions of kWh
# electricity_df_new_columns = ["County", "2013 (MWh)", "2014 (MWh)", "2015 (MWh)",
#                              "2016 (MWh)", "2017 (MWh)", "2018 (MWh)", "2019 (MWh)"]
# electricity_df.columns = electricity_df_new_columns
# electricity_df.head()

# ---------------------------------------------------------------------

# Cleaning up Environment Data Set

In [36]:
# Taking a look at environment_df
environment_df.sample(n=5)

,Unnamed: 0,County,Year,ETo (in),Precip (in),Sol Rad (Ly/day),Avg Vap Pres (mBars),Max Air Temp (F),Min Air Temp (F),Avg Air Temp (F),Max Rel Hum (%),Min Rel Hum (%),Avg Rel Hum (%),Dew Point (F),Avg Wind Speed (mph),Wind Run (miles),Avg Soil Temp (F)
65,65,Riverside,2019,672.13,98.83,473.761688,11.150334,123.7,23.4,67.205958,100.0,0.0,50.044884,45.434651,4.407507,105.776570,64.942880
105,105,Solano,2020,96.32,5.26,521.691262,12.150485,109.7,26.6,62.107767,100.0,1.0,63.726214,48.711262,6.924466,166.176893,60.666990
17,17,El Dorado,2018,113.58,59.43,447.856346,8.339749,100.0,20.0,59.942817,100.0,5.0,49.535565,38.491492,4.421618,106.096932,58.964156
11,11,Colusa,2018,52.74,9.27,406.634349,12.259834,103.2,21.8,61.293906,100.0,8.0,63.858726,47.936842,5.014681,120.304432,61.601108
51,51,Modoc,2020,43.67,5.94,499.772727,6.903030,99.5,3.5,51.036364,99.0,5.0,54.219697,33.562500,4.981061,119.561364,52.942045


In [37]:
# Drop "Unnamed: 0" column from environment_df
environment_df.drop("Unnamed: 0", axis=1, inplace=True)

In [38]:
# Format numbers in environment_df
environment_df["Sol Rad (Ly/day)"] = environment_df["Sol Rad (Ly/day)"].map("{:.0f}".format)
environment_df["Avg Vap Pres (mBars)"] = environment_df["Avg Vap Pres (mBars)"].map("{:.1f}".format)
environment_df["Avg Air Temp (F)"] = environment_df["Avg Air Temp (F)"].map("{:.1f}".format)
environment_df["Max Rel Hum (%)"] = environment_df["Max Rel Hum (%)"].map("{:.0f}".format)
environment_df["Min Rel Hum (%)"] = environment_df["Min Rel Hum (%)"].map("{:.0f}".format)
environment_df["Avg Rel Hum (%)"] = environment_df["Avg Rel Hum (%)"].map("{:.0f}".format)
environment_df["Dew Point (F)"] = environment_df["Dew Point (F)"].map("{:.1f}".format)
environment_df["Avg Wind Speed (mph)"] = environment_df["Avg Wind Speed (mph)"].map("{:.1f}".format)
environment_df["Wind Run (miles)"] = environment_df["Wind Run (miles)"].map("{:.1f}".format)
environment_df["Avg Soil Temp (F)"] = environment_df["Avg Soil Temp (F)"].map("{:.1f}".format)

environment_df.head()

,County,Year,ETo (in),Precip (in),Sol Rad (Ly/day),Avg Vap Pres (mBars),Max Air Temp (F),Min Air Temp (F),Avg Air Temp (F),Max Rel Hum (%),Min Rel Hum (%),Avg Rel Hum (%),Dew Point (F),Avg Wind Speed (mph),Wind Run (miles),Avg Soil Temp (F)
0,Alameda,2018,132.60,40.31,438,11.5,99.3,19.9,57.5,100,9,70,47.3,4.0,94.8,61.8
1,Alameda,2019,141.87,57.87,430,11.8,101.6,26.8,57.9,100,9,71,48.0,4.0,95.6,60.7
2,Alameda,2020,116.04,24.83,494,12.0,108.8,28.3,59.1,100,8,69,48.5,4.1,99.2,62.5
3,Alpine,2019,33.83,5.56,461,5.8,92.0,4.6,50.4,99,9,46,28.6,7.1,170.2,51.3
4,Alpine,2020,47.24,7.60,494,5.7,95.5,2.6,51.3,99,7,44,28.7,7.6,183.1,50.8


### Should Wind Run (miles) be cumulative like it is now (all the miles of wind run added up for each year for each county) or average? Asking because it's a huge number.

In [39]:
# Convert data types back into floats for formatted columns
environment_df["Sol Rad (Ly/day)"] = environment_df["Sol Rad (Ly/day)"].astype(float)
environment_df["Avg Vap Pres (mBars)"] = environment_df["Avg Vap Pres (mBars)"].astype(float)
environment_df["Avg Air Temp (F)"] = environment_df["Avg Air Temp (F)"].astype(float)
environment_df["Max Rel Hum (%)"] = environment_df["Max Rel Hum (%)"].astype(float)
environment_df["Min Rel Hum (%)"] = environment_df["Min Rel Hum (%)"].astype(float)
environment_df["Avg Rel Hum (%)"] = environment_df["Avg Rel Hum (%)"].astype(float)
environment_df["Dew Point (F)"] = environment_df["Dew Point (F)"].astype(float)
environment_df["Avg Wind Speed (mph)"] = environment_df["Avg Wind Speed (mph)"].astype(float)
environment_df["Wind Run (miles)"] = environment_df["Wind Run (miles)"].astype(float)
environment_df["Avg Soil Temp (F)"] = environment_df["Avg Soil Temp (F)"].astype(float)

In [40]:
# Check environment_df data types
environment_df.dtypes

County                   object
Year                      int64
ETo (in)                float64
Precip (in)             float64
Sol Rad (Ly/day)        float64
Avg Vap Pres (mBars)    float64
Max Air Temp (F)        float64
Min Air Temp (F)        float64
Avg Air Temp (F)        float64
Max Rel Hum (%)         float64
Min Rel Hum (%)         float64
Avg Rel Hum (%)         float64
Dew Point (F)           float64
Avg Wind Speed (mph)    float64
Wind Run (miles)        float64
Avg Soil Temp (F)       float64
dtype: object

In [41]:
# Check out environment_df
environment_df

,County,Year,ETo (in),Precip (in),Sol Rad (Ly/day),Avg Vap Pres (mBars),Max Air Temp (F),Min Air Temp (F),Avg Air Temp (F),Max Rel Hum (%),Min Rel Hum (%),Avg Rel Hum (%),Dew Point (F),Avg Wind Speed (mph),Wind Run (miles),Avg Soil Temp (F)
0,Alameda,2018,132.60,40.31,438.0,11.5,99.3,19.9,57.5,100.0,9.0,70.0,47.3,4.0,94.8,61.8
1,Alameda,2019,141.87,57.87,430.0,11.8,101.6,26.8,57.9,100.0,9.0,71.0,48.0,4.0,95.6,60.7
2,Alameda,2020,116.04,24.83,494.0,12.0,108.8,28.3,59.1,100.0,8.0,69.0,48.5,4.1,99.2,62.5
3,Alpine,2019,33.83,5.56,461.0,5.8,92.0,4.6,50.4,99.0,9.0,46.0,28.6,7.1,170.2,51.3
4,Alpine,2020,47.24,7.60,494.0,5.7,95.5,2.6,51.3,99.0,7.0,44.0,28.7,7.6,183.1,50.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,Yolo,2019,103.37,52.17,424.0,11.3,106.7,28.0,61.1,100.0,0.0,62.0,46.5,5.0,121.0,61.0
126,Yolo,2020,93.16,7.80,485.0,11.7,107.6,31.5,64.0,100.0,0.0,58.0,47.5,5.1,123.0,63.7
127,Yuba,2018,53.66,29.46,414.0,10.9,102.6,25.1,63.0,100.0,6.0,57.0,45.5,4.1,98.0,63.5
128,Yuba,2019,50.31,38.85,389.0,10.8,102.8,28.5,62.1,99.0,7.0,57.0,45.2,4.5,107.6,62.5


# ---------------------------------------------------------------------

# Matching all the Data Sets to Have the Same # of Counties

In [42]:
# See number of unique counties in fire_df
len(fire_df["County"].unique())

59

In [43]:
# List all the counties in fire_df to see which Counties are not in CA
fire_df["County"].unique().tolist()

['Tuolumne',
 'Los Angeles',
 'Riverside',
 'Placer',
 'Ventura',
 'Fresno',
 'Siskiyou',
 'Humboldt',
 'Tehama',
 'Shasta',
 'San Diego',
 'Kern',
 'Sonoma',
 'Contra Costa',
 'Butte',
 'Tulare',
 'Santa Barbara',
 'Mariposa',
 'Monterey',
 'El Dorado',
 'San Bernardino',
 'Plumas',
 'Modoc',
 'San Luis Obispo',
 'Madera',
 'Inyo',
 'Napa',
 'San Benito',
 'San Joaquin',
 'Lake',
 'Alameda',
 'Glenn',
 'Yolo',
 'Sacramento',
 'Stanislaus',
 'Solano',
 'Merced',
 'Mendocino',
 'Lassen',
 'Amador',
 'Yuba',
 'Nevada',
 'Santa Clara',
 'Calaveras',
 'San Mateo',
 'Orange',
 'Colusa',
 'Trinity',
 'Del Norte',
 'Mono',
 'Alpine',
 'Sutter',
 'Kings',
 'Sierra',
 'Santa Cruz',
 'Marin',
 'Mexico',
 'State of Oregon',
 'State of Nevada']

In [44]:
# Drop rows that have 'Mexico', 'State of Oregon', and 'State of Nevada' in fire_df counties
unwanted_county1_fire_df = fire_df[fire_df["County"] == "Mexico"].index
unwanted_county2_fire_df = fire_df[fire_df["County"] == "State of Oregon"].index
unwanted_county3_fire_df = fire_df[fire_df["County"] == "State of Nevada"].index

fire_df.drop(unwanted_county1_fire_df, inplace=True)
fire_df.drop(unwanted_county2_fire_df, inplace=True)
fire_df.drop(unwanted_county3_fire_df, inplace=True)

fire_df.head()

,Name,County,County IDs,Latitude,Longitude,Started,Year,Acres Burned,Cal Fire Incident,Major Incident
0,Rim Fire,Tuolumne,55,37.86,-120.09,2013-08-17 15:25:00+00:00,2013,257314,True,False
1,Powerhouse Fire,Los Angeles,19,34.59,-118.42,2013-05-30 15:28:00+00:00,2013,30274,True,False
2,Mountain Fire,Riverside,33,33.71,-116.73,2013-07-15 13:43:00+00:00,2013,27531,True,False
3,American Fire,Placer,31,39.12,-120.65,2013-08-10 16:30:00+00:00,2013,27440,False,False
4,Springs Fire,Ventura,56,0.00,0.00,2013-05-02 07:01:00+00:00,2013,24251,True,True


In [45]:
# Double-check to see if rows with data for "Mexico", "State of Oregon", and "State of Nevada" have been dropped
sorted(fire_df["County"].unique())

['Alameda',
 'Alpine',
 'Amador',
 'Butte',
 'Calaveras',
 'Colusa',
 'Contra Costa',
 'Del Norte',
 'El Dorado',
 'Fresno',
 'Glenn',
 'Humboldt',
 'Inyo',
 'Kern',
 'Kings',
 'Lake',
 'Lassen',
 'Los Angeles',
 'Madera',
 'Marin',
 'Mariposa',
 'Mendocino',
 'Merced',
 'Modoc',
 'Mono',
 'Monterey',
 'Napa',
 'Nevada',
 'Orange',
 'Placer',
 'Plumas',
 'Riverside',
 'Sacramento',
 'San Benito',
 'San Bernardino',
 'San Diego',
 'San Joaquin',
 'San Luis Obispo',
 'San Mateo',
 'Santa Barbara',
 'Santa Clara',
 'Santa Cruz',
 'Shasta',
 'Sierra',
 'Siskiyou',
 'Solano',
 'Sonoma',
 'Stanislaus',
 'Sutter',
 'Tehama',
 'Trinity',
 'Tulare',
 'Tuolumne',
 'Ventura',
 'Yolo',
 'Yuba']

In [46]:
# Now check to see the number of unique counties listed in fire_df
len(fire_df["County"].unique())

56

In [47]:
# See number of unique counties in housing_df
len(housing_df["County"].unique())

61

In [48]:
# List all the counties in housing_df to see which Counties are not in CA
housing_df["County"].unique().tolist()

['Humboldt',
 'Del Norte',
 nan,
 'Mendocino',
 'Trinity',
 'Sonoma',
 'Siskiyou',
 'Lake',
 'Marin',
 'Shasta',
 'Glenn',
 'Tehama',
 'Napa',
 'San Mateo',
 'San Francisco',
 'Colusa',
 'Contra Costa',
 'Alameda',
 'Solano',
 'Santa Cruz',
 'Yolo',
 'Santa Clara',
 'Butte',
 'Monterey',
 'Sutter',
 'Sacramento',
 'San Benito',
 'Yuba',
 'San Joaquin',
 'Placer',
 'Stanislaus',
 'Nevada',
 'Plumas',
 'Modoc',
 'San Luis Obispo',
 'Lassen',
 'El Dorado',
 'Merced',
 'Amador',
 'Sierra',
 'Calaveras',
 'Fresno',
 'Santa Barbara',
 'Tuolumne',
 'Madera',
 'Mariposa',
 'Kings',
 'Alpine',
 'Douglas',
 'Kern',
 'Tulare',
 'Mono',
 'Ventura',
 'Los Angeles',
 'Inyo',
 'Orange',
 'San Bernardino',
 'Riverside',
 'San Diego',
 'Imperial',
 'La Paz']

In [49]:
# Check to see if County column in housing_df is the only column with null values
housing_df.isna().sum()

County                 40
Latitude                0
Longitude               0
Ocean Proximity         0
Population              0
Households              0
Average Income          0
Average House Value     0
dtype: int64

In [50]:
# Drop rows that have 'La Paz', 'Douglas', and 'nan' in housing_df counties
unwanted_county1_housing_df = housing_df[housing_df["County"] == "La Paz"].index
unwanted_county2_housing_df = housing_df[housing_df["County"] == "Douglas"].index

housing_df.drop(unwanted_county1_housing_df, inplace=True)
housing_df.drop(unwanted_county2_housing_df, inplace=True)

housing_df = housing_df.dropna()

housing_df.head()

,County,Latitude,Longitude,Ocean Proximity,Population,Households,Average Income,Average House Value
0,Humboldt,40.54,-124.35,NEAR OCEAN,806,270,30147,94600
1,Del Norte,41.80,-124.30,NEAR OCEAN,1298,478,19797,85800
2,Del Norte,41.84,-124.30,NEAR OCEAN,1244,456,30313,103600
3,Humboldt,40.69,-124.27,NEAR OCEAN,1194,465,25179,79000
4,Humboldt,40.58,-124.26,NEAR OCEAN,907,369,23571,111400


In [51]:
# List all the counties in housing_df to see which Counties are not in CA
housing_df["County"].unique().tolist()

['Humboldt',
 'Del Norte',
 'Mendocino',
 'Trinity',
 'Sonoma',
 'Siskiyou',
 'Lake',
 'Marin',
 'Shasta',
 'Glenn',
 'Tehama',
 'Napa',
 'San Mateo',
 'San Francisco',
 'Colusa',
 'Contra Costa',
 'Alameda',
 'Solano',
 'Santa Cruz',
 'Yolo',
 'Santa Clara',
 'Butte',
 'Monterey',
 'Sutter',
 'Sacramento',
 'San Benito',
 'Yuba',
 'San Joaquin',
 'Placer',
 'Stanislaus',
 'Nevada',
 'Plumas',
 'Modoc',
 'San Luis Obispo',
 'Lassen',
 'El Dorado',
 'Merced',
 'Amador',
 'Sierra',
 'Calaveras',
 'Fresno',
 'Santa Barbara',
 'Tuolumne',
 'Madera',
 'Mariposa',
 'Kings',
 'Alpine',
 'Kern',
 'Tulare',
 'Mono',
 'Ventura',
 'Los Angeles',
 'Inyo',
 'Orange',
 'San Bernardino',
 'Riverside',
 'San Diego',
 'Imperial']

In [52]:
# Now check to see the number of unique counties listed in housing_df
len(housing_df["County"].unique())

58

In [59]:
# Create a copy of the housing_df called housing_df2
# In this new DF, drop Imperial and San Francisco county from it for Machine Learning Model
housing_df2 = housing_df.copy()

imperial_housing = housing_df2[housing_df2["County"] == "Imperial"].index
san_francisco_housing = housing_df2[housing_df2["County"] == "San Francisco"].index

housing_df2.drop(imperial_housing, inplace=True)
housing_df2.drop(san_francisco_housing, inplace=True)

housing_df2.head()

,County,Latitude,Longitude,Ocean Proximity,Population,Households,Average Income,Average House Value
0,Humboldt,40.54,-124.35,NEAR OCEAN,806,270,30147,94600
1,Del Norte,41.80,-124.30,NEAR OCEAN,1298,478,19797,85800
2,Del Norte,41.84,-124.30,NEAR OCEAN,1244,456,30313,103600
3,Humboldt,40.69,-124.27,NEAR OCEAN,1194,465,25179,79000
4,Humboldt,40.58,-124.26,NEAR OCEAN,907,369,23571,111400


In [60]:
# Check number of counties in housing_df2
len(housing_df2["County"].unique())

56

In [53]:
# See number of unique counties in electricity_df
len(electricity_df["County"].unique())

58

In [62]:
# Create a copy of the electricity_df called electricity_df2
# In this new DF, drop Imperial and San Francisco county from it for Machine Learning Model
electricity_df2 = electricity_df.copy()

imperial_electricity = electricity_df2[electricity_df2["County"] == "Imperial"].index
san_francisco_electricity = electricity_df2[electricity_df2["County"] == "San Francisco"].index

electricity_df2.drop(imperial_electricity, inplace=True)
electricity_df2.drop(san_francisco_electricity, inplace=True)

electricity_df2.head()

,County,Year,MWh
0,Alameda,2013,10618.66
1,Alpine,2013,18.18
2,Amador,2013,310.34
3,Butte,2013,1502.98
4,Calaveras,2013,326.66


In [63]:
# Check number of counties in electricity_df2
len(electricity_df2["County"].unique())

56

In [54]:
# List all the counties in electricity_df to see if they're CA counties
electricity_df["County"].unique().tolist()

['Alameda',
 'Alpine',
 'Amador',
 'Butte',
 'Calaveras',
 'Colusa',
 'Contra Costa',
 'Del Norte',
 'El Dorado',
 'Fresno',
 'Glenn',
 'Humboldt',
 'Imperial',
 'Inyo',
 'Kern',
 'Kings',
 'Lake',
 'Lassen',
 'Los Angeles',
 'Madera',
 'Marin',
 'Mariposa',
 'Mendocino',
 'Merced',
 'Modoc',
 'Mono',
 'Monterey',
 'Napa',
 'Nevada',
 'Orange',
 'Placer',
 'Plumas',
 'Riverside',
 'Sacramento',
 'San Benito',
 'San Bernardino',
 'San Diego',
 'San Francisco',
 'San Joaquin',
 'San Luis Obispo',
 'San Mateo',
 'Santa Barbara',
 'Santa Clara',
 'Santa Cruz',
 'Shasta',
 'Sierra',
 'Siskiyou',
 'Solano',
 'Sonoma',
 'Stanislaus',
 'Sutter',
 'Tehama',
 'Trinity',
 'Tulare',
 'Tuolumne',
 'Ventura',
 'Yolo',
 'Yuba']

In [55]:
# See number of unique counties in environment_df
len(environment_df["County"].unique())

44

In [56]:
# List all the counties in environment_df to see which Counties are missing
environment_df["County"].unique().tolist()

['Alameda',
 'Alpine',
 'Amador',
 'Butte',
 'Colusa',
 'Contra Costa',
 'El Dorado',
 'Fresno',
 'Humboldt',
 'Imperial',
 'Inyo',
 'Kern',
 'Kings',
 'Los Angeles',
 'Marin',
 'Mendocino',
 'Merced',
 'Modoc',
 'Monterey',
 'Napa',
 'Orange',
 'Placer',
 'Riverside',
 'Sacramento',
 'San Benito',
 'San Bernardino',
 'San Diego',
 'San Joaquin',
 'San Luis Obispo',
 'San Mateo',
 'Santa Barbara',
 'Santa Clara',
 'Santa Cruz',
 'Shasta',
 'Siskiyou',
 'Solano',
 'Sonoma',
 'Stanislaus',
 'Sutter',
 'Tehama',
 'Tulare',
 'Ventura',
 'Yolo',
 'Yuba']

In [57]:
environment_df.head()

,County,Year,ETo (in),Precip (in),Sol Rad (Ly/day),Avg Vap Pres (mBars),Max Air Temp (F),Min Air Temp (F),Avg Air Temp (F),Max Rel Hum (%),Min Rel Hum (%),Avg Rel Hum (%),Dew Point (F),Avg Wind Speed (mph),Wind Run (miles),Avg Soil Temp (F)
0,Alameda,2018,132.60,40.31,438.0,11.5,99.3,19.9,57.5,100.0,9.0,70.0,47.3,4.0,94.8,61.8
1,Alameda,2019,141.87,57.87,430.0,11.8,101.6,26.8,57.9,100.0,9.0,71.0,48.0,4.0,95.6,60.7
2,Alameda,2020,116.04,24.83,494.0,12.0,108.8,28.3,59.1,100.0,8.0,69.0,48.5,4.1,99.2,62.5
3,Alpine,2019,33.83,5.56,461.0,5.8,92.0,4.6,50.4,99.0,9.0,46.0,28.6,7.1,170.2,51.3
4,Alpine,2020,47.24,7.60,494.0,5.7,95.5,2.6,51.3,99.0,7.0,44.0,28.7,7.6,183.1,50.8


# ---------------------------------------------------------------------

# Export Housing CSV, Electricity CSV, and Fire CSV
## Once Environment Data has all counties ** Except ** Imperial and San Francisco, then we can merge ALL Datasets and export as CSV

In [64]:
# Export all the CSVs
housing_df2.to_csv("housing_ml_model.csv")
electricity_df2.to_csv("electricity_ml_model.csv")
fire_df.to_csv("fire_ml_model.csv")